In [7]:
import csv
import numpy as np

with open('../.idea/po_słowach_integers-po_slowach-matrix-pl-20211020', 'r', encoding='UTF-8') as file:
    matrix_raw = file.readlines()
    del matrix_raw[0]
    matrix = []
    article_id = []
    for row in matrix_raw:
        splitted_row = row.split()
        article_id.append(splitted_row[0])
        del splitted_row[0]
        matrix.append(splitted_row)

with open('../.idea/po_słowach_integers-po_slowach-cats_dict-pl-20211020', 'r', encoding='UTF-8') as file:
    categories_raw = file.readlines()
    categories = []
    categories_id = []
    for category in categories_raw:
        categories.append(category.split('\t')[0])
        categories_id.append(category.split('\t')[1])
    print(categories)

#with open('../.idea/po_słowach_integers-po_slowach-categories-pl-20211020', 'r', encoding='UTF-8') as file:
#    categories_articles = file.readlines()

with open('../.idea/po_słowach_integers-po_slowach-categories-pl-20211020', 'r', encoding='UTF-8') as file:
    data_iter = csv.reader(file, delimiter = '\t')
    data = [data for data in data_iter]
categories_articles = np.asarray(data, dtype=np.int32)

with open('../.idea/po_słowach_integers-po_slowach-articles_dict-pl-20211020', 'r', encoding='UTF-8') as file:
    articles = file.readlines()

['Bułgarscy_satyrycy', 'Obchody_rocznicowe', 'Polskie_filmy_o_podróżach_w_czasie', 'Radiestezja', 'Rozrywki_umysłowe', 'Upamiętnienie_Świętego_Wojciecha', 'Akcesoria_do_palenia', 'Naukowe_projekty_meteorologiczne', 'Albumy_Janet_Jackson', 'Motocykle_turystyczne']


In [30]:
from sklearn.model_selection import train_test_split, cross_val_score
X = np.array(matrix, dtype=np.int32)
y = categories_articles[:, 1]
X_train, X_test, y_train, y_test = train_test_split(X, y)



In [31]:
#SVM (LinearSVC)

from sklearn.svm import LinearSVC
m_LinearSVC = LinearSVC(tol=1.0e-6,max_iter=5000,verbose=1)
#m_LinearSVC.fit(X_train, y_train)
#results = m_LinearSVC.predict(X_test)
cross_val_score(m_LinearSVC, X, y, cv=5, scoring='recall_macro')

[LibLinear][LibLinear][LibLinear][LibLinear][LibLinear]

array([0.95, 0.95, 1.  , 1.  , 0.9 ])

In [34]:
#KNN

from sklearn.neighbors import KNeighborsClassifier
m_KNN = KNeighborsClassifier()
cross_val_score(m_KNN, X, y, cv=5, scoring='recall_macro')

array([0.7 , 0.45, 0.55, 0.55, 0.45])

In [30]:
from numpy import genfromtxt, int32
my_data = genfromtxt('../.idea/po_słowach_integers-po_slowach-matrix-pl-20211020', dtype = int32, skip_header = 1, delimiter=' ')
bag_of_words = dict()

for row in my_data:
    row_dict = dict()
    for i, word_count in enumerate(row[1:], start = 1):
        if word_count > 0:
            row_dict[i] = word_count
    bag_of_words[row[0]] = row_dict
